In [19]:
import io
import numpy as np
import pandas as pd
import requests
from pprint import pprint
from tqdm import tqdm

def ades_GetPointEau(numéro_département: int=None, nb_réseau: int=None):
    """Récupérer et formater les données"""
    url = 'https://ades.eaufrance.fr/Recherche/GetPointEau'
    req_data = {
        'type': '1',
        'filter': f'departement={numéro_département}&nbReseau={nb_réseau}',
    }
    response_GetPointEau = requests.post(url=url, data=req_data)
    point_eau_liste = response_GetPointEau.json()['pointEaux']
    # Formater données retournées par fonction ades_GetPointEau
    point_eau_liste_formatée = []
    for p in point_eau_liste:
        dic = {}
        dic['coordinates0'] = p['Coordinates'][0]
        dic['coordinates1'] = p['Coordinates'][1]
        dic['coord_com'] = p['Properties']['coord_com']
        dic['coord_ref0'] = p['Properties']['coord_ref'][0]
        dic['coord_ref1'] = p['Properties']['coord_ref'][1]
        dic['epsg_ref'] = p['Properties']['epsg_ref']
        dic['exist_mesures'] = p['Properties']['exist_mesures']
        if ' ' in p['Properties']['label']:
            dic['Identifiant BSS'] = p['Properties']['label'].split(' ')[0]
            dic['Ancien code national BSS'] = p['Properties']['label'].split(' ')[1].replace('(', '').replace(')', '')
        else:
            dic['Identifiant BSS'] = p['Properties']['label']
            dic['Ancien code national BSS'] = 'N/D'
        dic['position_captage'] = p['Properties']['position_captage']
        dic['type_pt_eau'] = p['Properties']['type_pt_eau']
        point_eau_liste_formatée.append(dic)
        return point_eau_liste_formatée


def ades_GetPointEauIPSDataCSV(numéro_département=None, nombre_points=None):
    """Retourne le contenu d'un fichier CSV sous forme d'octets"""
    url = 'https://ades.eaufrance.fr/Recherche/GetPointEauIPSDataCSV'
    query_params = {
        'filters': f'departement={numéro_département}',
        'nombre_points': nombre_points,
        'typeRecherche': 1,
    }
    response_GetPointEauIPSDataCSV = requests.get(url=url, params=query_params)
    content_as_bytes = response_GetPointEauIPSDataCSV.content
    return content_as_bytes


def ades_FicheMesureTabDonnees(row):
    url = 'https://ades.eaufrance.fr/Fiche/MesureTabDonnees'
    post_params = {
        'CodeBSS': row['Ancien code national BSS'],
        'DateDebut': 0,
        'DateFin': row['Date de fin'],
        'length': row['Nb de mesures'],
    }
    response = requests.post(url, data=post_params)
    noms_colonnes = ['Date', 'Cote piézo. (mNGF)', "Mode d'obtention", 'Nature de la mesure', 'Qualification', 'Statut', 'Producteur']
    nature_mesure_par_défaut = 'Naturel'
    df_FicheMesureTabDonnees = pd.DataFrame(response.json()['data'], columns=noms_colonnes)
    df_FicheMesureTabDonnees.loc[:, 'Cote piézo. (mNGF)'] = df_FicheMesureTabDonnees['Cote piézo. (mNGF)'].str.replace(',', '.')
    df_FicheMesureTabDonnees.loc[:, 'Cote piézo. (mNGF)'] = df_FicheMesureTabDonnees['Cote piézo. (mNGF)'].astype(dtype=np.float64)
    df_FicheMesureTabDonnees.loc[df_FicheMesureTabDonnees['Nature de la mesure'] == '',
                                 'Nature de la mesure'] = nature_mesure_par_défaut
    df_FicheMesureTabDonnees['Ancien code national BSS'] = row['Ancien code national BSS']
    df_FicheMesureTabDonnees['Identifiant BSS'] = row['Identifiant BSS']
    return df_FicheMesureTabDonnees


def get_mesure_tab_donnees_for_every_point_eau(df_point_eau: pd.DataFrame) -> pd.DataFrame:
    points_eau_liste = df_point_eau.to_dict(orient='records')
    row0 = points_eau_liste[0]
    df_full = pd.merge(left=ades_FicheMesureTabDonnees(row0),
                        right=pd.DataFrame([row0]),
                        on=['Ancien code national BSS', 'Identifiant BSS'],
                        how='outer')
    for row in tqdm(points_eau_liste[56:]):
        df_tmp = pd.merge(left=ades_FicheMesureTabDonnees(row),
                        right=pd.DataFrame([row]),
                        on=['Ancien code national BSS', 'Identifiant BSS'],
                        how='outer')
        df_full = pd.concat([df_full, df_tmp])
    return df_full

numéro_département = 66

point_eau_liste = ades_GetPointEau(numéro_département, nb_réseau=0)

point_eau_ips_data_csv_bytes = ades_GetPointEauIPSDataCSV(numéro_département, 
                                                          nombre_points=len(point_eau_liste))

df_point_eau = pd.merge(left=pd.DataFrame(point_eau_liste),
                        right=pd.read_csv(io.BytesIO(point_eau_ips_data_csv_bytes), sep=';'),
                        on=['Ancien code national BSS', 'Identifiant BSS'],
                        how='outer')

df_ades_full = get_mesure_tab_donnees_for_every_point_eau(df_point_eau)
df_ades_full

 10%|█         | 1/10 [00:00<00:06,  1.43it/s]


JSONDecodeError: Expecting value: line 3 column 1 (char 4)

```bash
curl 'https://ades.eaufrance.fr/Fiche/MesureTabDonnees' -X POST \
-H 'User-Agent: Mozilla/5.0 (Windows NT 10.0; Win64; x64; rv:109.0) Gecko/20100101 Firefox/114.0' \
-H 'Accept: application/json, text/javascript, */*; q=0.01' \
-H 'Accept-Language: en-US,en;q=0.7,fr-FR;q=0.3' \
-H 'Accept-Encoding: gzip, deflate, br' \
-H 'Content-Type: application/x-www-form-urlencoded; charset=UTF-8' \
-H 'X-Requested-With: XMLHttpRequest' \
-H 'Origin: https://ades.eaufrance.fr' \
-H 'DNT: 1' \
-H 'Connection: keep-alive' \
-H 'Referer: https://ades.eaufrance.fr/Fiche/PtEau?Code=10795X0001/S' \
-H 'Cookie: ASP.NET_SessionId=5h0ea5equ1dgpv1afo1o35ex; BIGipServerpool_ades_public_https=2517346496.47873.0000; TS019555ba=01ce77d600f0ec24bf8bf34d88a0bed770976920a8752e1fde4e397e7cf6a95af60b1f2794a590960ea8f71d99724bd0e747659886cd4e82b42619bbb624df6cce836125fc70d1f8d01e84c595bc868a511e7cd854' \
-H 'Sec-Fetch-Dest: empty' \
-H 'Sec-Fetch-Mode: cors' \
-H 'Sec-Fetch-Site: same-origin' \
--data-raw 'draw=1&columns%5B0%5D%5Bdata%5D=0&columns%5B0%5D%5Bname%5D=&columns%5B0%5D%5Bsearchable%5D=true&columns%5B0%5D%5Borderable%5D=true&columns%5B0%5D%5Bsearch%5D%5Bvalue%5D=&columns%5B0%5D%5Bsearch%5D%5Bregex%5D=false&columns%5B1%5D%5Bdata%5D=1&columns%5B1%5D%5Bname%5D=&columns%5B1%5D%5Bsearchable%5D=true&columns%5B1%5D%5Borderable%5D=true&columns%5B1%5D%5Bsearch%5D%5Bvalue%5D=&columns%5B1%5D%5Bsearch%5D%5Bregex%5D=false&columns%5B2%5D%5Bdata%5D=2&columns%5B2%5D%5Bname%5D=&columns%5B2%5D%5Bsearchable%5D=true&columns%5B2%5D%5Borderable%5D=true&columns%5B2%5D%5Bsearch%5D%5Bvalue%5D=&columns%5B2%5D%5Bsearch%5D%5Bregex%5D=false&columns%5B3%5D%5Bdata%5D=3&columns%5B3%5D%5Bname%5D=&columns%5B3%5D%5Bsearchable%5D=true&columns%5B3%5D%5Borderable%5D=true&columns%5B3%5D%5Bsearch%5D%5Bvalue%5D=&columns%5B3%5D%5Bsearch%5D%5Bregex%5D=false&columns%5B4%5D%5Bdata%5D=4&columns%5B4%5D%5Bname%5D=&columns%5B4%5D%5Bsearchable%5D=true&columns%5B4%5D%5Borderable%5D=true&columns%5B4%5D%5Bsearch%5D%5Bvalue%5D=&columns%5B4%5D%5Bsearch%5D%5Bregex%5D=false&columns%5B5%5D%5Bdata%5D=5&columns%5B5%5D%5Bname%5D=&columns%5B5%5D%5Bsearchable%5D=true&columns%5B5%5D%5Borderable%5D=true&columns%5B5%5D%5Bsearch%5D%5Bvalue%5D=&columns%5B5%5D%5Bsearch%5D%5Bregex%5D=false&columns%5B6%5D%5Bdata%5D=6&columns%5B6%5D%5Bname%5D=&columns%5B6%5D%5Bsearchable%5D=true&columns%5B6%5D%5Borderable%5D=true&columns%5B6%5D%5Bsearch%5D%5Bvalue%5D=&columns%5B6%5D%5Bsearch%5D%5Bregex%5D=false&order%5B0%5D%5Bcolumn%5D=0&order%5B0%5D%5Bdir%5D=asc&start=0&length=2000&search%5Bvalue%5D=&search%5Bregex%5D=false&CodeBSS=10795X0001%2FS&DateDebut=27%2F02%2F2002&DateFin=11%2F01%2F2011&Couples=3&KnowNbResult=false'
```

In [82]:
from urllib.parse import unquote
s = 'draw=1&columns%5B0%5D%5Bdata%5D=0&columns%5B0%5D%5Bname%5D=&columns%5B0%5D%5Bsearchable%5D=true&columns%5B0%5D%5Borderable%5D=true&columns%5B0%5D%5Bsearch%5D%5Bvalue%5D=&columns%5B0%5D%5Bsearch%5D%5Bregex%5D=false&columns%5B1%5D%5Bdata%5D=1&columns%5B1%5D%5Bname%5D=&columns%5B1%5D%5Bsearchable%5D=true&columns%5B1%5D%5Borderable%5D=true&columns%5B1%5D%5Bsearch%5D%5Bvalue%5D=&columns%5B1%5D%5Bsearch%5D%5Bregex%5D=false&columns%5B2%5D%5Bdata%5D=2&columns%5B2%5D%5Bname%5D=&columns%5B2%5D%5Bsearchable%5D=true&columns%5B2%5D%5Borderable%5D=true&columns%5B2%5D%5Bsearch%5D%5Bvalue%5D=&columns%5B2%5D%5Bsearch%5D%5Bregex%5D=false&columns%5B3%5D%5Bdata%5D=3&columns%5B3%5D%5Bname%5D=&columns%5B3%5D%5Bsearchable%5D=true&columns%5B3%5D%5Borderable%5D=true&columns%5B3%5D%5Bsearch%5D%5Bvalue%5D=&columns%5B3%5D%5Bsearch%5D%5Bregex%5D=false&columns%5B4%5D%5Bdata%5D=4&columns%5B4%5D%5Bname%5D=&columns%5B4%5D%5Bsearchable%5D=true&columns%5B4%5D%5Borderable%5D=true&columns%5B4%5D%5Bsearch%5D%5Bvalue%5D=&columns%5B4%5D%5Bsearch%5D%5Bregex%5D=false&columns%5B5%5D%5Bdata%5D=5&columns%5B5%5D%5Bname%5D=&columns%5B5%5D%5Bsearchable%5D=true&columns%5B5%5D%5Borderable%5D=true&columns%5B5%5D%5Bsearch%5D%5Bvalue%5D=&columns%5B5%5D%5Bsearch%5D%5Bregex%5D=false&columns%5B6%5D%5Bdata%5D=6&columns%5B6%5D%5Bname%5D=&columns%5B6%5D%5Bsearchable%5D=true&columns%5B6%5D%5Borderable%5D=true&columns%5B6%5D%5Bsearch%5D%5Bvalue%5D=&columns%5B6%5D%5Bsearch%5D%5Bregex%5D=false&order%5B0%5D%5Bcolumn%5D=0&order%5B0%5D%5Bdir%5D=asc&start=0&length=2000&search%5Bvalue%5D=&search%5Bregex%5D=false&CodeBSS=10795X0001%2FS&DateDebut=27%2F02%2F2002&DateFin=11%2F01%2F2011&Couples=3&KnowNbResult=false'
query_params = [tuple(qp.split('=')) for qp in unquote(s).split('&')]
query_params

[('draw', '1'),
 ('columns[0][data]', '0'),
 ('columns[0][name]', ''),
 ('columns[0][searchable]', 'true'),
 ('columns[0][orderable]', 'true'),
 ('columns[0][search][value]', ''),
 ('columns[0][search][regex]', 'false'),
 ('columns[1][data]', '1'),
 ('columns[1][name]', ''),
 ('columns[1][searchable]', 'true'),
 ('columns[1][orderable]', 'true'),
 ('columns[1][search][value]', ''),
 ('columns[1][search][regex]', 'false'),
 ('columns[2][data]', '2'),
 ('columns[2][name]', ''),
 ('columns[2][searchable]', 'true'),
 ('columns[2][orderable]', 'true'),
 ('columns[2][search][value]', ''),
 ('columns[2][search][regex]', 'false'),
 ('columns[3][data]', '3'),
 ('columns[3][name]', ''),
 ('columns[3][searchable]', 'true'),
 ('columns[3][orderable]', 'true'),
 ('columns[3][search][value]', ''),
 ('columns[3][search][regex]', 'false'),
 ('columns[4][data]', '4'),
 ('columns[4][name]', ''),
 ('columns[4][searchable]', 'true'),
 ('columns[4][orderable]', 'true'),
 ('columns[4][search][value]', ''),
